In [17]:
import os 
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go

> SOMETHING IS MESSED UP

# Classifier

## Collect data

In [18]:
parameters_to_keep = [
    'classifier_hiddenlayer_dim',
    'embeddingmodel_output',
    'model_train_classifier_learning_rate',
    'model_train_classifier_momentum',
    'model_train_classifier_weight_decay',
    'model_train_embedding_learning_rate',
    'model_train_embedding_momentum',
    'model_train_embedding_weight_decay',
    'model_train_n_epoch',
]

results = {
    'classifier_hiddenlayer_dim' : [],
    'embeddingmodel_output' : [],
    'model_train_classifier_learning_rate' : [],
    'model_train_classifier_momentum' : [],
    'model_train_classifier_weight_decay' : [],
    'model_train_embedding_learning_rate' : [],
    'model_train_embedding_momentum' : [],
    'model_train_embedding_weight_decay' : [],
    'model_train_n_epoch' : [],

    'min_valid_loss' : [],
    'test_f1_max' : [],
    'train_f1_max' : [],
    'valid_f1_max' : [],
    'filename' : []
}

folder = './save_random_search_2'

for current_dir in os.listdir(folder):
    with open(f'{folder}/{current_dir}/config.json', 'r') as file:
        config = json.load(file)

    results["filename"].append(f'{folder}/{current_dir}')
    for key in parameters_to_keep:
        results[key].append(config[key])

    results["min_valid_loss"].append(
        min(pd.read_csv(f'{folder}/{current_dir}/loss_train.csv')['0'])
    )

    metrics_file = pd.read_csv(f'{folder}/{current_dir}/metrics_save.csv')
    results["test_f1_max"].append(
        max(metrics_file.loc[metrics_file["loop"] == "test",'f1'])
    )
    results["train_f1_max"].append(
        max(metrics_file.loc[metrics_file["loop"] == "train",'f1'])
    )
    results["valid_f1_max"].append(
        max(metrics_file.loc[metrics_file["loop"] == "validation",'f1'])
    )

results = pd.DataFrame(results)

# plot Data

In [19]:
results.head()

,classifier_hiddenlayer_dim,embeddingmodel_output,model_train_classifier_learning_rate,model_train_classifier_momentum,model_train_classifier_weight_decay,model_train_embedding_learning_rate,model_train_embedding_momentum,model_train_embedding_weight_decay,model_train_n_epoch,min_valid_loss,test_f1_max,train_f1_max,valid_f1_max,filename
0,250,pooler_output,0.000001,0.55,0.004,0.00020,None,0.075,5,1.035215,0.349372,0.355329,0.401455,./save_random_search_2/61
1,200,last_hidden_state,0.010000,0.50,0.001,0.00005,None,0.075,5,0.688727,0.440807,0.507434,0.516865,./save_random_search_2/95
2,150,pooler_output,0.010000,0.60,0.004,0.00020,None,0.100,5,1.066637,0.369874,0.357961,0.400794,./save_random_search_2/59
3,100,pooler_output,0.010000,0.50,0.004,0.00040,None,0.050,5,1.066163,0.356647,0.343816,0.390008,./save_random_search_2/92
4,250,last_hidden_state,0.005000,0.50,0.001,0.00040,None,0.050,5,1.065125,0.329696,0.347961,0.369213,./save_random_search_2/66


### Learning rate

In [20]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
        "categoryorder" : "median ascending",
       "title" : "Learning Rate"
    },
   "yaxis":{"title" : "F1-Score (max)"},
   "title" : {"text" : "Classifier : Impact of Learning rate on the F1-score"},
})
fig.add_trace( go.Box(
                x = results["model_train_classifier_learning_rate"], 
                y = results["test_f1_max"],
                xaxis='x',
                name = "Test"
))
fig.add_trace( go.Box(
                x = results["model_train_classifier_learning_rate"], 
                y = results["valid_f1_max"],
                xaxis='x',
                name = "Validation"
))

# fig.add_trace( go.Box(
#                 x = results["model_train_classifier_learning_rate"], 
#                 y = results["train_f1_max"],
#                 xaxis='x',
#                 name = "Train"
# ))
fig

In [21]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
       "categoryorder" : "median ascending",
       "title" : "Learning Rate"
    },
   "yaxis":{"title" : "Validation Loss Value (min)"},
   "title" : {"text" : "Classifier : Impact of Learning rate on the Minimum validation loss"},
})
fig.add_trace( go.Box(
                x = results["model_train_classifier_learning_rate"], 
                y = results["min_valid_loss"],
                xaxis='x',
                name = "Validation"
))

fig

### Momentum

In [22]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
        # "categoryorder" : "median ascending",
       "title" : "Momentum"
    },
   "yaxis":{"title" : "F1-Score (max)"},
   "title" : {"text" : "Classifier : Impact of momentum on the F1-score"},
})
fig.add_trace( go.Box(
                x = results["model_train_classifier_momentum"], 
                y = results["test_f1_max"],
                xaxis='x',
                name = "Test"
))
fig.add_trace( go.Box(
                x = results["model_train_classifier_momentum"], 
                y = results["valid_f1_max"],
                xaxis='x',
                name = "Validation"
))

# fig.add_trace( go.Box(
#                 x = results["model_train_classifier_learning_rate"], 
#                 y = results["train_f1_max"],
#                 xaxis='x',
#                 name = "Train"
# ))
fig

In [23]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
        # "categoryorder" : "median ascending",
       "title" : "Momentum"
    },
   "yaxis":{"title" : "Validation Loss Value (min)"},
   "title" : {"text" : "Classifier : Impact of momentum on the Minimum validation loss"},
})
fig.add_trace( go.Box(
                x = results["model_train_classifier_momentum"], 
                y = results["min_valid_loss"],
                xaxis='x',
                name = "Validation"
))

fig

### Weight decay NEEDS TO BE REMADE WITH  LESS VALUES

In [24]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "category",
        "categoryorder" : "median ascending",
       "title" : "Classifier : Weight decay"
    },
   "yaxis":{"title" : "F1-Score (max)"},
   "title" : {"text" : "Impact of weight decay on the F1-score"},
})
fig.add_trace( go.Box(
                x = results["model_train_classifier_weight_decay"], 
                y = results["test_f1_max"],
                xaxis='x',
                name = "Test"
))
fig.add_trace( go.Box(
                x = results["model_train_classifier_weight_decay"], 
                y = results["valid_f1_max"],
                xaxis='x',
                name = "Validation"
))

# fig.add_trace( go.Box(
#                 x = results["model_train_classifier_learning_rate"], 
#                 y = results["train_f1_max"],
#                 xaxis='x',
#                 name = "Train"
# ))
fig

In [25]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "category",
        "categoryorder" : "median ascending",
       "title" : "Classifier : Weight decay"
    },
   "yaxis":{"title" : "Validation Loss Value (min)"},
   "title" : {"text" : "Impact of weight decay on the Minimum validation loss"},
})
fig.add_trace( go.Box(
                x = results["model_train_classifier_weight_decay"], 
                y = results["min_valid_loss"],
                xaxis='x',
                name = "Validation"
))

fig

### Hidden layer dimension (Needs additional values)

In [26]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
        # "categoryorder" : "median ascending",
       "title" : "Classifier : Hidden layer dimension"
    },
   "yaxis":{"title" : "F1-Score (max)"},
   "title" : {"text" : "Impact of the dimension of the hidden layer on the F1-score"},
})
fig.add_trace( go.Box(
                x = results["classifier_hiddenlayer_dim"], 
                y = results["test_f1_max"],
                xaxis='x',
                name = "Test"
))
fig.add_trace( go.Box(
                x = results["classifier_hiddenlayer_dim"], 
                y = results["valid_f1_max"],
                xaxis='x',
                name = "Validation"
))

# fig.add_trace( go.Box(
#                 x = results["model_train_classifier_learning_rate"], 
#                 y = results["train_f1_max"],
#                 xaxis='x',
#                 name = "Train"
# ))
fig

In [27]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
        # "categoryorder" : "median ascending",
       "title" : "Classifier : Hidden layer dimension"
    },
   "yaxis":{"title" : "Validation Loss Value (min)"},
   "title" : {"text" : "Impact of the dimension of the hidden layer on the Minimum validation loss"},
})
fig.add_trace( go.Box(
                x = results["classifier_hiddenlayer_dim"], 
                y = results["min_valid_loss"],
                xaxis='x',
                name = "Validation"
))

fig

# Embedder

## Plot Data

### Learning rate

In [28]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
        # "categoryorder" : "median ascending",
       "title" : "Learning Rate"
    },
   "yaxis":{"title" : "F1-Score (max)"},
   "title" : {"text" : "Embedder : Impact of Learning rate on the F1-score"},
})
fig.add_trace( go.Box(
                x = results["model_train_embedding_learning_rate"], 
                y = results["test_f1_max"],
                xaxis='x',
                name = "Test"
))
fig.add_trace( go.Box(
                x = results["model_train_embedding_learning_rate"], 
                y = results["valid_f1_max"],
                xaxis='x',
                name = "Validation"
))

# fig.add_trace( go.Box(
#                 x = results["model_train_classifier_learning_rate"], 
#                 y = results["train_f1_max"],
#                 xaxis='x',
#                 name = "Train"
# ))
fig

In [29]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "log",
        # "categoryorder" : "median ascending",
       "title" : "Learning Rate"
    },
   "yaxis":{"title" : "Validation Loss Value (min)"},
   "title" : {"text" : "Embedder : Impact of Learning rate on the Minimum validation loss"},
})
fig.add_trace( go.Box(
                x = results["model_train_embedding_learning_rate"], 
                y = results["min_valid_loss"],
                xaxis='x',
                name = "Validation"
))

fig

### Weight decay

In [30]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "category",
      #  "categoryorder" : "median ascending",
       "title" : "Weight decay"
    },
   "yaxis":{"title" : "F1-Score (max)"},
   "title" : {"text" : "Embedder : Impact of weight decay on the F1-score"},
})
fig.add_trace( go.Box(
                x = results["model_train_embedding_weight_decay"], 
                y = results["test_f1_max"],
                xaxis='x',
                name = "Test"
))
fig.add_trace( go.Box(
                x = results["model_train_embedding_weight_decay"], 
                y = results["valid_f1_max"],
                xaxis='x',
                name = "Validation"
))

# fig.add_trace( go.Box(
#                 x = results["model_train_classifier_learning_rate"], 
#                 y = results["train_f1_max"],
#                 xaxis='x',
#                 name = "Train"
# ))
fig

In [31]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "category",
        "categoryorder" : "median ascending",
       "title" : "Weight decay"
    },
   "yaxis":{"title" : "Validation Loss Value (min)"},
   "title" : {"text" : "Embedder : Impact of weight decay on the Minimum validation loss"},
})
fig.add_trace( go.Box(
                x = results["model_train_embedding_weight_decay"], 
                y = results["min_valid_loss"],
                xaxis='x',
                name = "Validation"
))

fig

### Embedding output

In [32]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "category",
      #   "categoryorder" : "median ascending",
       "title" : "Embedding ouptut"
    },
   "yaxis":{"title" : "F1-Score (max)"},
   "title" : {"text" : "Embedder : Impact of embedding output on the F1-score"},
})
fig.add_trace( go.Box(
                x = results["embeddingmodel_output"], 
                y = results["test_f1_max"],
                xaxis='x',
                name = "Test"
))
fig.add_trace( go.Box(
                x = results["embeddingmodel_output"], 
                y = results["valid_f1_max"],
                xaxis='x',
                name = "Validation"
))

# fig.add_trace( go.Box(
#                 x = results["model_train_classifier_learning_rate"], 
#                 y = results["train_f1_max"],
#                 xaxis='x',
#                 name = "Train"
# ))
fig

In [33]:
fig = go.Figure(layout = {
   "xaxis":{
       "domain" : [0,1],
       "type" : "category",
        # "categoryorder" : "median ascending",
       "title" : "Learning Rate"
    },
   "yaxis":{"title" : "Validation Loss Value (min)"},
   "title" : {"text" : "Embedder : Impact of Learning rate on the Minimum validation loss"},
})
fig.add_trace( go.Box(
                x = results["embeddingmodel_output"], 
                y = results["min_valid_loss"],
                xaxis='x',
                name = "Validation"
))

fig